In [115]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, ELU, Add, Softmax, LayerNormalization, LayerNormalization, MultiHeadAttention
from tensorflow.keras.models import Model
import numpy as np
import tensorflow.keras as keras
import importlib
from tqdm import tqdm
import os
from importlib import reload
reload(models)
from models import * 

In [116]:
import models
import dataprocessor
importlib.reload(models)
importlib.reload(dataprocessor)
from models import *
from dataprocessor import *

macd_timescales = {(8, 24), (16, 28), (32, 96)}
rtn_timescales = {1, 21, 63, 126, 252}
timesteps = 16

folder_path = 'data'
files = [f[:-5] for f in os.listdir(folder_path) if f.endswith('.xlsx')]
files = [file for file in files if file not in ("CC00.NYB", "LB00.CME", "ES00.CME", "NQ00.CME", "YM00.CBT")]
data_list = []

for file in tqdm(files, desc="处理文件中。。"):
    data = pd.read_excel(f"data/{file}.xlsx")[["日期", "收盘价(元)"]]
    data = data.rename(columns={"日期": "date", "收盘价(元)": "close"}).sort_values("date")
    data["side_info"] = file
    try:
        data = generate_features(data, macd_timescales, rtn_timescales)
    except:
        print(file)
    data_list.append(data)

print("Process data into a dataset")
targetset = generate_tf_dataset(data_list, timesteps)
contextset = generate_tf_dataset(data_list, timesteps, True)

处理文件中。。: 100%|██████████| 12/12 [00:33<00:00,  2.78s/it]


Process data into a dataset


In [117]:
s = encode_context_info(contextset)

x = []
for features, _ in contextset:
    x.append(features.numpy())

x = x[:100]
x = tf.constant(x)
s = s[:100]

sequence_length = 9
encoding_size = 12
hidden_dim = 8

model = BaselineNeuralForecaster(sequence_length, hidden_dim, encoding_size)
model.build((x.shape, s.shape))
V = model(x, s)

编码中...


2024-06-20 22:35:48.954139: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-20 22:35:56.199041: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [118]:
s = encode_context_info(targetset)

x = []
for features, _ in targetset:
    x.append(features.numpy())

x = x[:100]
x = tf.constant(x)
s = s[:100]

sequence_length = 8
encoding_size = 12 
hidden_dim = 8

model = BaselineNeuralForecaster(sequence_length, hidden_dim, encoding_size)
model.build((x.shape, s.shape))
K = model(x, s)


编码中...


2024-06-20 22:36:03.196496: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-20 22:36:10.673461: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [132]:
model = AttentionWrapper(hidden_dim, 4, sequence_length, encoding_size)
input_shape = ((V.shape, K.shape, x.shape, s.shape))
model.build(input_shape)
y = model(V, K, s, x)

In [133]:
y

<tf.Tensor: shape=(100, 16, 8), dtype=float32, numpy=
array([[[-0.7058372 ,  0.268996  , -0.21691963, ...,  2.066096  ,
          0.05795419,  0.6882519 ],
        [-0.7056248 ,  0.26882666, -0.21708634, ...,  2.0660071 ,
          0.05814856,  0.6883856 ],
        [-0.7059304 ,  0.26919276, -0.21681195, ...,  2.0661376 ,
          0.05790943,  0.6880081 ],
        ...,
        [-0.7055423 ,  0.26890212, -0.21716088, ...,  2.0659788 ,
          0.05824864,  0.68827623],
        [-0.7055667 ,  0.2689979 , -0.21714476, ...,  2.0659924 ,
          0.05823958,  0.6881719 ],
        [-0.7055083 ,  0.26900566, -0.2172085 , ...,  2.065971  ,
          0.05829561,  0.6881617 ]],

       [[-0.71909446,  0.30384213, -0.24336475, ...,  2.0663078 ,
          0.07304496,  0.6593779 ],
        [-0.71938705,  0.30422366, -0.24315804, ...,  2.06643   ,
          0.07283551,  0.6589975 ],
        [-0.7193807 ,  0.3043738 , -0.2431986 , ...,  2.0664306 ,
          0.0728634 ,  0.6588461 ],
        ...,
